In [1]:
import random
import utils
import numpy as np
import matplotlib.pyplot as plt
from Game.LeducGame import *
from Environment.Environment import *
from Opponent_agent.RandAgent import *
from Opponent_agent.GreedyAgent import *
from Opponent_agent.HumanAgent import *
from Qagent.QAgent import *
from numpy import asarray
from numpy import savetxt

stack_size=5
random_agent_qtable=None
greedy_agent_qtable=None

# Training and Testing parameters

In [2]:
epochs_number=500
evaluate_every=5000
test_number=10000

## qAgent vs GreedyAgent

In [3]:

greedy=GreedyAgent()
env=Environment(greedy)
envTest=Environment(greedy)
qagent=QAgent()


for i in range(epochs_number):  
    
    #TRAIN
    #print("NEWWWWWW")
    while(env.game.epoch_is_over==0):
        
        current_player=env.game.get_firstplayer()
        stack1=env.game.stack1
        stack2=env.game.stack2
        #print("stack1: ", stack1)
        #print("stack2: ", stack2)
        if(env.game.stack1<= env.game.stack2):
            small_stack=env.game.stack1
        else:
            small_stack=env.game.stack2
        #print("small_stack: ", small_stack)    
        

        allowed_actions=[0,1,2]
        if(current_player==1):
            env.agent.set_action(allowed_actions,None,env.game.get_game_round(), env.game.get_hand_player2(),env.game.get_boardcard())
            env.set_opponent_action(env.agent.get_action())
            reward,current_player,allowed_actions=env.game.step_prime(env.agent.get_action(), small_stack)
            
            #print("opponent action: ", env.agent.get_action())

        while(env.game.is_game_over()==0):
            #print("\n NEW ")
            #print("qagent card: ", env.game.hand_player1)
            #print("board action: ", env.game.boardcard)
            #print("round : ", env.game.game_round)
            state=env.get_state()
            qagent.set_state(state)
            #print("State= ",state)
            qagent_action=qagent.explore_action(allowed_actions)
            #print("qagent action: ", qagent_action)

            reward,allowed_actions, new_state=env.step(qagent_action, small_stack)
            

            if(env.game.is_game_over()==1):
                #print("results: ",env.game.result)
                qagent.update(reward, env.get_actions_hist(),stack1,stack2)
                
        env.reset_card()
        state=env.get_state()
        qagent.set_state(state)

    env.reset()
    state=env.get_state()
    qagent.set_state(state)

with np.printoptions(precision=3, suppress=True):
    print(qagent.qtable)
greedy_agent_qtable=qagent.qtable

#savetxt('data.csv', data, delimiter=',')
#np.savetxt('array.csv', greedy_agent_qtable, delimiter=',', fmt='%d')
with open('test.npy', 'wb') as f:
    np.save(f, greedy_agent_qtable)
with open('test.npy', 'rb') as f:
    a = np.load(f)
print(a)

[[[-1.72   1.355  4.221]
  [-2.343  1.299  3.344]
  [-2.848  2.184  3.454]
  [-2.343  1.445  1.576]
  [-2.848  0.     2.301]
  [-3.063  2.433  0.   ]
  [-1.72   0.     1.346]
  [-2.048  0.     1.649]
  [-0.95   0.     1.966]
  [ 0.     0.     0.   ]
  [ 0.     0.     0.4  ]
  [ 0.     0.5    0.6  ]
  [ 0.     0.6    0.   ]
  [ 0.     0.6    0.   ]
  [-0.5    0.     0.   ]
  [-0.95   0.6    0.6  ]
  [-0.95   0.5    1.264]
  [-0.95   0.6    0.6  ]
  [ 0.     0.     0.   ]
  [ 0.     0.     0.   ]
  [ 0.     0.4    0.   ]
  [ 0.     0.     0.   ]
  [ 0.     0.     0.   ]
  [-0.95   0.     0.   ]
  [ 0.     0.5    0.6  ]
  [ 0.     1.04   0.   ]
  [ 0.     0.     0.   ]
  [ 0.     0.     0.   ]
  [ 0.     0.     0.4  ]
  [ 0.     0.     0.4  ]
  [ 0.     0.     0.   ]
  [ 0.     0.     0.   ]
  [-0.5    0.     0.   ]
  [ 0.     0.     0.   ]
  [ 0.     0.     0.76 ]
  [ 0.     0.     0.   ]]

 [[ 2.87   4.576  4.445]
  [ 3.085  4.431  4.002]
  [ 2.45   3.682  5.148]
  [ 3.4    3.588  2.752

In [4]:
kk
from matplotlib import cm
import numpy as np

tab = pd.DataFrame(qagent.qtable[5])
tab_n = tab.div(tab.max(axis=1), axis=0)
ax = sns.heatmap(tab_n,cmap='viridis')

NameError: name 'kk' is not defined

In [ ]:
with np.printoptions(precision=3, suppress=True):
    print(qagent.qtable[0])

In [ ]:
import pandas as pd
import seaborn as sns

df = pd.DataFrame(qagent.qtable[3], columns=['fold', "check","push"])

cm = sns.light_palette('blue', as_cmap=True)

s = df.style.background_gradient(cmap=cm, low=0, high=1, axis=1)
s

In [ ]:
import matplotlib.pyplot as plt

# generate some example data
matrix = np.random.uniform(0,1,(5,5))

# plot the matrix as an image with an appropriate colormap
plt.imshow(qagent.qtable[1].T, aspect='auto', cmap="bwr")

# add the values
for (i, j), value in np.ndenumerate(qagent.qtable[1]):
    plt.text(i, j, "%.3f"%value, va='center', ha='center')

plt.axis('off')
plt.show()

## qAgent (trained with a greedy agent) vs HumanAgent

In [ ]:
humanAgent=HumanAgent()
env=Environment(humanAgent)
qagent=QAgent()

with open('test.npy', 'rb') as f:
    a = np.load(f)
qagent.set_qtable(a)
print("How to play?\n\nFor the actions:\n\t- 0 : Fold\n\t- 1 : Check\n\t- 2 : Push\n\nFor the cards:\n\t- 0 : Jack\n\t- 1 : Queen\n\t- 2 : King\n\n ")
while(env.game.epoch_is_over==0):
    #env.display()
    print("\n==NEW GAME==")
    current_player=env.game.get_firstplayer()
    stack1=env.game.stack1
    stack2=env.game.stack2
    #print("stack1: ", stack1)
    #print("stack2: ", stack2)
    if(env.game.stack1<= env.game.stack2):
        small_stack=env.game.stack1
    else:
        small_stack=env.game.stack2
    #print("small_stack: ", small_stack)    


    allowed_actions=[0,1,2]

    #IF HUMAN AGENT
    if(current_player==1):
        print("Your stack = ",env.game.stack2)
        print("QAgent stack = ",env.game.stack1)
        env.agent.set_action(allowed_actions,None,env.game.get_game_round(), env.game.get_hand_player2(),env.game.get_boardcard())
        env.set_opponent_action(env.agent.get_action())
        reward,current_player,allowed_actions=env.game.step_prime(env.agent.get_action(), small_stack)
        if(env.game.is_game_over()==1):
            print("You loose!! He has a:", env.game.get_hand_player1(), "\n\n")

        #print("opponent action: ", env.agent.get_action())

    while(env.game.is_game_over()==0):
        #print("\n NEW ")
        #print("qagent card: ", env.game.hand_player1)
        #print("board action: ", env.game.boardcard)
        #print("round : ", env.game.game_round)
        state=env.get_state()
        qagent.set_state(state)
        #print("State= ",state)
        qagent_action=qagent.exploit_action(allowed_actions, env.game.stack1)
        #print("qagent action: ", qagent_action)

        reward,allowed_actions, new_state=env.step_human(qagent_action, small_stack)


        if(env.game.is_game_over()==1):
            print("results: ",env.game.result)
            if(reward<0):
                print("His action is: ", qagent_action, "\nHe has a: ",env.game.get_hand_player1(), "\nCongratulation, you win !!!\n\n" )
            elif(reward>0):
                print("His action is: ", qagent_action, "\nHe has a: ",env.game.get_hand_player1(), "\nSorry, you loose !!!\n\n" )
            elif(reward==0):
                print("His action is: ", qagent_action, "\nHe has a: ",env.game.get_hand_player1(), "\nIt's a draw !!!\n\n" )

    env.reset_card()
    state=env.get_state()
    qagent.set_state(state)

env.reset()
state=env.get_state()
qagent.set_state(state)

In [ ]:
print(qagent.qtable[4])

In [ ]:
humanAgent=HumanAgent()
env=Environment(humanAgent)
qagent=QAgent()
qagent.set_qtable(greedy_agent_qtable)

    
print("How to play?\n\nFor the actions:\n\t- 0 : Fold\n\t- 1 : Check\n\t- 2 : Push\n\nFor the cards:\n\t- 0 : Jack\n\t- 1 : Queen\n\t- 2 : King\n\n ")
for i in range(1):
    allowed_actions=[0,1,2]
    current_player=env.game.get_firstplayer()
    while(env.game.is_game_over()==0):
        state=env.get_state()
        qagent.set_state(state)
        
        if (current_player==1):
            env.agent.set_action(allowed_actions,None, env.game.get_game_round(), env.game.get_hand_player2(),env.game.get_boardcard())
            env.set_opponent_action(env.agent.get_action())
            reward,current_player,allowed_actions=env.game.step_prime(env.agent.get_action())
            if(env.game.is_game_over()==1):
                print("You loose!! He has a:", env.game.get_hand_player1(), "\n\n")
        if(env.game.is_game_over()==0):
            state=env.get_state()
            qagent.set_state(state)
            qagent_action=qagent.exploit_action(allowed_actions)
            reward,allowed_actions, new_state=env.step(qagent_action)
            if(env.game.is_game_over()==1):
                if(reward<0):
                    print("His action was: ", qagent_action, "\nHe has a: ",env.game.get_hand_player1(), "\nCongratulation, you win !!!\n\n" )
                elif(reward>0):
                    print("His action was: ", qagent_action, "\nHe has a: ",env.game.get_hand_player1(), "\nSorry, you loose !!!\n\n" )
                elif(reward==0):
                    print("His action was: ", qagent_action, "\nHe has a: ",env.game.get_hand_player1(), "\nIt's a draw !!!\n\n" )
                qagent.update(reward, env.get_actions_hist())
            qagent.set_state(new_state)

    
    env.reset()
    